In [2]:
from datasets import load_dataset
dataset = load_dataset("stanfordnlp/imdb")
train_data = dataset["train"]
test_data = dataset["test"]

# train_data[0]
print(test_data[0])

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [3]:
import requests

API_KEY = "952110236cee7e960053e51148f2c511"
movie_title = "Top Gun: Maverick"

# Search for movie ID
search_url = f"https://api.themoviedb.org/3/search/movie?api_key={API_KEY}&query={movie_title}"
response = requests.get(search_url).json()
print(response)
# movie_id = response["results"][0]["id"]

# Get cast and crew
# credits_url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={API_KEY}"
# credits = requests.get(credits_url).json()

# Extract names
# actors = [member["name"] for member in credits["cast"][:5]]
# directors = [member["name"] for member in credits["crew"] if member["job"] == "Director"]

# print("Actors:", actors)
# print("Director:", directors)


{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/kBSSbN1sOiJtXjAGVZXxHJR9Kox.jpg', 'genre_ids': [28, 18], 'id': 361743, 'original_language': 'en', 'original_title': 'Top Gun: Maverick', 'overview': 'After more than thirty years of service as one of the Navy’s top aviators, and dodging the advancement in rank that would ground him, Pete “Maverick” Mitchell finds himself training a detachment of TOP GUN graduates for a specialized mission the likes of which no living pilot has ever seen.', 'popularity': 15.2438, 'poster_path': '/62HCnUTziyWcpDaBO2i1DX17ljH.jpg', 'release_date': '2022-05-21', 'title': 'Top Gun: Maverick', 'video': False, 'vote_average': 8.2, 'vote_count': 10317}, {'adult': False, 'backdrop_path': '/l1GmkC9EJLDVRt8RxT64cet6QUQ.jpg', 'genre_ids': [99, 10770], 'id': 1076135, 'original_language': 'fr', 'original_title': 'Top Gun Maverick : Le phénomène', 'overview': '36 years after the release of the first "Top Gun", this sequel has been eagerly awaited by fans aro

In [ ]:
import spacy

